In [1]:
from qiskit import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')
from qiskit.ignis.characterization.gates import *
from qiskit.providers.aer.noise.errors.standard_errors import coherent_unitary_error
from qiskit.providers.aer.noise import NoiseModel


C:\Users\Annanay\Anaconda3\envs\project\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


https://qiskit.org/documentation/stubs/qiskit.qobj.PulseQobjInstruction.html?highlight=pulseqobjinstruction#qiskit.qobj.PulseQobjInstruction

In [2]:
def ampcal_circuits(max_reps, qubits, th, phi, lam):

    #change this as required to *2
    xdata = np.arange(max_reps)*1

    qr = qiskit.QuantumRegister(max(qubits)+1)
    cr = qiskit.ClassicalRegister(len(qubits))
    circuits = []
    for circ_index, circ_length in enumerate(xdata):
        circ = qiskit.QuantumCircuit(qr, cr)
        circ.name = 'ampcal1Qcircuit_' + str(circ_index) + '_0'
        for qind, qubit in enumerate(qubits):
            #circ.u2(-np.pi/2, np.pi/2, qr[qubit])
            for _ in range(circ_length):
                circ.barrier(qr[qubit])
                circ.u3(th, phi, lam, qr[qubit])

        for qind, qubit in enumerate(qubits):
            circ.measure(qr[qubit], cr[qind])
        circuits.append(circ)

    return circuits, xdata

In [3]:
def ampcal_u2_circuits(max_reps, qubits, phi, lam):

    #change this as required to *2
    xdata = np.arange(max_reps)*1

    qr = qiskit.QuantumRegister(max(qubits)+1)
    cr = qiskit.ClassicalRegister(len(qubits))
    circuits = []
    for circ_index, circ_length in enumerate(xdata):
        circ = qiskit.QuantumCircuit(qr, cr)
        circ.name = 'ampcal1Qcircuit_' + str(circ_index) + '_0'
        for qind, qubit in enumerate(qubits):
            #circ.u2(-np.pi/2, np.pi/2, qr[qubit])
            for _ in range(circ_length):
                circ.barrier(qr[qubit])
                circ.u2(phi,lam, qr[qubit])

        for qind, qubit in enumerate(qubits):
            circ.measure(qr[qubit], cr[qind])
        circuits.append(circ)

    return circuits, xdata

In [4]:
max_reps = 300
qubits = [0]
circs_x, xdata = ampcal_circuits(max_reps, qubits, np.pi, 0, np.pi)
circs_r180, xdata = ampcal_circuits(max_reps, qubits, np.pi, -1*np.pi/2, np.pi/2)
circs_r360, xdata = ampcal_circuits(max_reps, qubits, 2*np.pi, -1*np.pi/2, np.pi/2)

In [5]:
circs_x90, xdata = ampcal_u2_circuits(max_reps, qubits, -np.pi/2, np.pi/2)
circs_y90, xdata = ampcal_u2_circuits(max_reps, qubits, 0, 0)

In [6]:
backends = ['ibmqx2', 'ibmq_ourense', 'ibmq_valencia', 'ibmq_16_melbourne', 'ibmq_vigo']
backends = list(map(lambda x: provider.get_backend(x), backends))

In [7]:
#executing in this way ensures we max out the queues and don't
#get anything cancelled!
job = {i: [qiskit.execute(circs_x[75*j:75*(j+1)],backend=i,shots=8192) for j in range(4)] for i in backends}
result_x = {i: list(map(lambda x: x.result().get_counts(), job[i])) for i in backends}

In [8]:
job = {i: [qiskit.execute(circs_r360[75*j:75*(j+1)],backend=i,shots=8192) for j in range(4)] for i in backends}
result_r360 = {i: list(map(lambda x: x.result().get_counts(), job[i])) for i in backends}
job = {i: [qiskit.execute(circs_r180[75*j:75*(j+1)],backend=i,shots=8192) for j in range(4)] for i in backends}
result_r180 = {i: list(map(lambda x: x.result().get_counts(), job[i])) for i in backends}
job = {i: [qiskit.execute(circs_x90[75*j:75*(j+1)],backend=i,shots=8192) for j in range(4)] for i in backends}
result_x90 = {i: list(map(lambda x: x.result().get_counts(), job[i])) for i in backends}
job = {i: [qiskit.execute(circs_y90[75*j:75*(j+1)],backend=i,shots=8192) for j in range(4)] for i in backends}
result_y90 = {i: list(map(lambda x: x.result().get_counts(), job[i])) for i in backends}

In [19]:
result_r360

{'ibmqx2': [{'0': 7634, '1': 558},
  {'0': 7679, '1': 513},
  {'0': 7681, '1': 511},
  {'0': 7713, '1': 479},
  {'0': 7671, '1': 521},
  {'0': 7676, '1': 516},
  {'0': 7662, '1': 530},
  {'0': 7664, '1': 528},
  {'0': 7693, '1': 499},
  {'0': 7668, '1': 524},
  {'0': 7689, '1': 503},
  {'0': 7661, '1': 531},
  {'0': 7696, '1': 496},
  {'0': 7630, '1': 562},
  {'0': 7675, '1': 517},
  {'0': 7706, '1': 486},
  {'0': 7661, '1': 531},
  {'0': 7690, '1': 502},
  {'0': 7680, '1': 512},
  {'0': 7682, '1': 510},
  {'0': 7692, '1': 500},
  {'0': 7693, '1': 499},
  {'0': 7710, '1': 482},
  {'0': 7680, '1': 512},
  {'0': 7686, '1': 506},
  {'0': 7659, '1': 533},
  {'0': 7667, '1': 525},
  {'0': 7697, '1': 495},
  {'0': 7679, '1': 513},
  {'0': 7674, '1': 518},
  {'0': 7719, '1': 473},
  {'0': 7674, '1': 518},
  {'0': 7692, '1': 500},
  {'0': 7728, '1': 464},
  {'0': 7662, '1': 530},
  {'0': 7682, '1': 510},
  {'0': 7693, '1': 499},
  {'0': 7643, '1': 549},
  {'0': 7715, '1': 477},
  {'0': 7717, '

In [13]:
result_r360 = dict(zip(list(map(lambda x: x.name(), result_r360.keys())), result_r360.values()))
result_r180 = dict(zip(list(map(lambda x: x.name(), result_r180.keys())), result_r180.values()))
result_x = dict(zip(list(map(lambda x: x.name(), result_x.keys())), result_x.values()))
result_x90 = dict(zip(list(map(lambda x: x.name(), result_x90.keys())), result_x90.values()))
result_y90 = dict(zip(list(map(lambda x: x.name(), result_y90.keys())), result_y90.values()))

In [16]:
from functools import reduce
for i in result_r360.keys():
    result_r360[i] = reduce(lambda x,y: x+y,result_r360[i])
    result_r180[i] = reduce(lambda x,y: x+y,result_r180[i])
    result_x90[i] = reduce(lambda x,y: x+y,result_x90[i])
    result_y90[i] = reduce(lambda x,y: x+y,result_y90[i])
    result_x[i] = reduce(lambda x,y: x+y,result_x[i])

In [25]:
for i in result_r360.keys():
    result_r360[i] = list(map(lambda x: x['0'],result_r360[i]))
    result_r180[i] = list(map(lambda x: x['0'],result_r180[i]))
    result_x90[i] = list(map(lambda x: x['0'],result_x90[i]))
    result_y90[i] = list(map(lambda x: x['0'],result_y90[i]))
    result_x[i] = list(map(lambda x: x['0'],result_x[i]))

In [29]:
pd.DataFrame(result_r360).to_csv('result_r360.csv')
pd.DataFrame(result_r180).to_csv('result_r180.csv')
pd.DataFrame(result_x90).to_csv('result_x90.csv')
pd.DataFrame(result_y90).to_csv('result_y90.csv')
pd.DataFrame(result_x).to_csv('result_x.csv')

In [ ]:
#Only run this cell for retrieving jobs.
k=100
job_x = [*backend.jobs(limit = k, job_name='sigma_x_1')[0].result().get_counts(),
         *backend.jobs(limit = k, job_name='sigma_x_2')[0].result().get_counts()]
job_r = [*backend.jobs(limit = k, job_name='x_180_1')[0].result().get_counts(),
         *backend.jobs(limit = k, job_name='x_180_2')[0].result().get_counts()]
job_ry90 = [*backend.jobs(limit = k, job_name='y_90_1')[0].result().get_counts(),
            *backend.jobs(limit = k, job_name='y_90_2')[0].result().get_counts()]
job_rx90 = [*backend.jobs(limit = k, job_name='x_90_1')[0].result().get_counts(),
            *backend.jobs(limit = k, job_name='x_90_2')[0].result().get_counts()]
#Only for 4 qubit 
job_ampcal = [*backend.jobs(limit = k, job_name='ampcal_1')[0].result().get_counts(),
             *backend.jobs(limit = k, job_name='ampcal_2')[0].result().get_counts(),
             *backend.jobs(limit = k, job_name='ampcal_3')[0].result().get_counts(),
             *backend.jobs(limit = k, job_name='ampcal_4')[0].result().get_counts()]
job_ampcal = list(map(lambda x: x['1']/8192, job_ampcal))
job_x = list(map(lambda x: x['1']/8192, job_x))
job_r = list(map(lambda x: x['1']/8192, job_r))
job_rx90 = list(map(lambda x: x['1']/8192, job_rx90))
job_ry90 = list(map(lambda x: x['1']/8192, job_ry90))

In [ ]:
from scipy.optimize import curve_fit

def f(x, A, B): # this is your 'straight line' y=f(x)
    return A*x + B
x = xdata[::2]
y = job_r[::2]
popt, pcov = curve_fit(f, x,y) # your data x, y to fit

residuals = y- f(x, *popt)
ss_res = np.sum(residuals**2)
ss_tot = np.sum((y-np.mean(y))**2)
r_squared = 1 - (ss_res / ss_tot)


In [ ]:
fig, ax = plt.subplots()
plt.scatter(xdata, job_x, color='g', label='sigma_x')
plt.scatter(xdata, job_r, color='b', label = 'rxpi')
plt.scatter(xdata[:75],job_rx90[::2], color='r', label='rx90')
plt.scatter(xdata[:75],job_ry90[::2], color='k', label='ry90')
plt.title('ibmq_armonk')
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()
plt.scatter(xdata, job_x, color='g', label='sigma_x')
plt.scatter(xdata, job_r, color='b', label = 'rxpi')
plt.scatter(xdata[:75],job_rx90[1::2], color='r', label='rx90')
plt.scatter(xdata[:75],job_ry90[1::2], color='k', label='ry90')
plt.title('ibmq_armonk starting at 1')
plt.legend()
plt.show()

In [ ]:
dict(zip(np.fft.fftfreq(len(fft)), list(map(np.abs, fft))))

In [ ]:
%matplotlib inline
# Fit the data to an oscillation

plt.figure(figsize=(15, 9))
initial_theta = 0.02
initial_c = 0.5
initial_phi = 0.1


fit = AmpCalFitter(backend_result, xdata, qubits,
                   fit_p0=[initial_theta, initial_c],
                   fit_bounds=([-np.pi, -1],
                               [np.pi, 1]))

fit.plot(0, ax=plt.gca())

#plot analytic function

print("Rotation Error on U2: %f rads"%(fit.angle_err()[0]))
plt.show()

In [ ]:
backend = provider.get_backend('ibmqx2')

In [ ]:
circ = [0]*75
for i in range(75):
    circ[i] = QuantumCircuit(2,2)
    circ[i].x(0)
    for n in range(i):
        circ[i].cx(0,1)
    circ[i].measure([0,1],[0,1])

shots = 8192
backend_result2 = qiskit.execute(circ, backend, shots=shots).result()

In [ ]:
circ[0].draw(output='mpl')

In [ ]:
res = backend_result2.get_counts()
content = {'both 0': [i['00'] for i in res], 'control 1, other 0': [i['01'] for i in res],'control 0, other 1': [i['10'] for i in res],'both 1': [i['11'] for i in res]}
pd.DataFrame(content).to_csv('ibmqx2_cx.csv')

In [ ]:
# Fit the data to an oscillation

plt.figure(figsize=(10, 6))
initial_theta = 0.02
initial_c = 0.5
initial_phi = 0.01


fit = AmpcalCalFitter(backend_result2, xdata, qubits,
                   fit_p0=[initial_theta, initial_c],
                   fit_bounds=([-np.pi, -1],
                               [np.pi, 1]))

fit.plot(0, ax=plt.gca())
print ("Angle error between X and Y: " + str(fit.angle_err()[0]))

plt.show()
